In [6]:
from filler import fill_redacted_text, evaluate_redacted
import json
import tqdm
import time
import pandas as pd

In [7]:
data = pd.read_csv("inputs/radiology.csv")

data.head()
print(len(data))

1000


In [8]:
notes = data["text"]
notes.head()

0    CT CHEST WITHOUT CONTRAST\n\nREASON FOR EXAM: ...
1    MEDICAL HISTORY:  ___ woman with hemolytic ane...
2    INDICATION:  ___ year old man with metastatic ...
3    INDICATION:  ___ man with right-sided testicul...
4    EXAMINATION:  CT LOWER EXT W/C LEFT\n\nINDICAT...
Name: text, dtype: object

In [ ]:
notes = notes.sample(500)
notes

825    Abdominal fluoro without radiologist.  Gastric...
892    EXAMINATION:  CHEST (PORTABLE AP)\n\nINDICATIO...
658    EXAMINATION:  CT ABD AND PELVIS W/O CONTRAST\n...
338    EXAMINATION:  MR HEAD W AND W/O CONTRAST\n\nIN...
565    EXAMINATION:  LIVER OR GALLBLADDER US (SINGLE ...
                             ...                        
320    INDICATION:  Endocarditis with vegetation foun...
404    CHEST RADIOGRAPH\n\nINDICATION: To look for pu...
793    EXAMINATION:  Bilateral DIGITAL 2D SCREENING M...
108    HISTORY:  ___ woman, with known intracranial b...
155    EXAMINATION:  L-SPINE (AP AND LAT)\n\nINDICATI...
Name: text, Length: 500, dtype: object

In [ ]:
quota=500

In [ ]:
filled_data = []
total = 0

for note in tqdm.tqdm(notes):
    retries = 0
    while retries < 3:
        try:
            redacted = fill_redacted_text(note)
            evaluate_report = evaluate_redacted(json.dumps(redacted, indent=2))  
            
            if evaluate_report["passed"]:
                filled_text = redacted["mapping_text"]
                redacted_text = redacted["mapping_text"]
                
                for pii_token in redacted["pii_mappings"]:
                    filled_text = filled_text.replace(pii_token, redacted["pii_mappings"][pii_token]["value"])
                    
                filled_data.append({
                    "original_text": note,
                    "filled_text": filled_text,
                    "redacted_text": redacted["mapping_text"],
                    "pii_mappings": redacted["pii_mappings"],
                })
                
                total += 1
                break
            else:
                print(f"Failed to fill the note: {note}")
                print(evaluate_report["reasoning"])
                time.sleep(1)
            
        except Exception as e:
            retries += 1
            print(f"Error: {e}")
            time.sleep(1)

json.dump(filled_data, open("outputs/radiology_filled.json", "w"))
print(f"Generated {total} filled notes")

  0%|          | 0/500 [00:00<?, ?it/s]

Failed to fill the note: Abdominal fluoro without radiologist.  Gastric band felt under fluoro, 24
seconds of fluoro.  No images were submitted.

The input contains no PII placeholders (no '___' or PII tokens like <<PII_1>>), yet the pii_mappings dictionary is empty. Based on the evaluation criteria, this suggests the PII replacement process was not properly initiated or the original text formatting was incorrect.
Failed to fill the note: Abdominal fluoro without radiologist.  Gastric band felt under fluoro, 24
seconds of fluoro.  No images were submitted.

The original text contains no "___" placeholders to be replaced with PII tokens, and the pii_mappings dictionary is empty. Since there are no PII tokens (<<PII_1>>, <<PII_2>>, etc.) in mapping_text, there are no corresponding entries needed in pii_mappings, making this technically consistent but incomplete for PII generation purposes.
Failed to fill the note: Abdominal fluoro without radiologist.  Gastric band felt under fluoro, 24


  1%|          | 5/500 [00:18<20:34,  2.49s/it]  

Failed to fill the note: EXAMINATION:  CT SINUS W/O CONTRAST FOR SURGICAL PLANNING Q114 CT HEADSINUS

INDICATION:  Dizziness, lightheadedness, nasal blockage.// Dizziness,
lightheadedness, nasal ___  AUTH# ___ ___

TECHNIQUE:  Axial images were acquired through the paranasal sinuses.  Bone
and soft tissue reconstructed images were generated. Coronal reformatted
images were then produced.

DOSE:  Acquisition sequence:
   1) Spiral Acquisition 3.0 s, 22.3 cm; CTDIvol = 27.3 mGy (Head) DLP = 609.1
mGy-cm.
 Total DLP (Head) = 609 mGy-cm.

COMPARISON:  None.

FINDINGS: 

Minimal mucosal thickening bilateral inferior frontal sinuses, patent right,
mildly narrowed left frontal sinus drainage pathway..

Minimal volume loss right maxillary sinus consistent with atelectasis.  Trace
mucosal thickening bilateral maxillary sinuses, small volume fluid within both
sinuses posteriorly.  Mild narrowing bilateral infundibula of maxillary sinus
ostiomeatal units by mucosal thickening.

Mild opacification

  3%|▎         | 15/500 [00:35<19:23,  2.40s/it]


KeyboardInterrupt: 